In [ ]:
#This Notebook is there to split retro and pro cohorts, 
# to create cases and controls 
# and to prepare for ML pipeline 

In [ ]:
import numpy as np
import pandas as pd
import sys

# Loading needed DataFrames

# load Case & Controls Complications MRNS

# unique are MRNs that have been diagnosed after HT_onset & for the first time
#Controls are the ones that have never had the disease
# all together CR= 59.606, Heart = 51.544, Renal = 59.950

All3 = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/All3_diseases/Cases_all3_cohorts_after_HT_onset.pkl")
All3_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/All3_diseases/Controls_No_all3_cohorts_Disease_Ever.pkl")

Cerebro = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Cerebrovascular_Diseases/Cases_CR_after_HT_onset.pkl")
Cerebro_Controls = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Cerebrovascular_Diseases/Controls_No_Cerebrovascular_Diseases_Ever.pkl')
   
Heart = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/Cases_heart_disease_after_HT_onset.pkl")
Heart_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/Controls_No_Heart_Diseases_Ever.pkl")

Renal = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Renal_Diseases/Cases_renal_disease_after_HT_onset.pkl")
Renal_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Renal_Diseases/Controls_No_Renal_Diseases_Ever.pkl")


df_final = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int_with_extra_BP.pkl')



In [ ]:
def prepare_df_for_split(df_Cases, df_Controls, df_final):
    
    df_Cases = df_Cases.reset_index('medical_record_number')
    df_Cases = df_Cases.sort_values('onset_year',ascending=True).drop_duplicates('medical_record_number', keep = 'first')
    df_Cases['Complication'] = '1'
    
    df_Controls = df_Controls[['HT_onset_year','date_of_birth_actual','age_in_days_HT_onset']]
    df_Controls['Complication'] = '0'
    df_Controls= df_Controls.reset_index('medical_record_number')
    
    Cohort = pd.concat([df_Cases, df_Controls], sort=True)
    Cohort= Cohort.set_index('medical_record_number')
    
    Train = Cohort.loc[Cohort['HT_onset_year'] < 2012]
    Test = Cohort.loc[Cohort['HT_onset_year'] >= 2012]
    
    Train['train_test'] = 'train'
    Test['train_test'] = 'test'
    
    Cohort = pd.concat([Train, Test], sort=True )
    
    df_final = df_final.set_index('medical_record_number')
    Cohort_Final = pd.merge(df_final, Cohort, on = 'medical_record_number', how = 'inner')
     
    return Cohort_Final    


In [ ]:
#setting the Cohort that we need

df = prepare_df_for_split(Renal,Renal_Controls, df_final)
df

In [ ]:
#df.to_csv('Check_ICD_Codes.csv')

In [ ]:
#df.loc[df['Complication'] == '1']
#df.loc[df['train_test'] == 'test']
#df.loc[df['train_test'] == 'train']

df.loc[(df['train_test'] == 'test') & (df['Complication'] == '1')]

In [ ]:
#df.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/Real_Retro_Pro_Split/Heart_Retro_Pro_Split_with_all_unsupervised_merged_cleaned_features.pkl')


In [ ]:
def drop_columns_for_ML_pipeline(df):
    
    df = df.drop(columns=['HT_onset_year','age_in_days', 'age_in_days_HT_onset', 'age_in_days_delta',
                 'context_diagnosis_code', 'context_name', 'date_of_birth_actual', 'onset_year'])
    df = df.reset_index('medical_record_number')
    return df

In [ ]:
df = drop_columns_for_ML_pipeline(df)
df

In [ ]:
for col in df:
    print (col)

In [ ]:
df.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2012/Without_demographic_data/with_extra_BP/Renal_with_extra_BP.pkl')

